## Important Libraries and Modules

In [48]:
import pandas as pd
# from gradcam import GradCAM
import tensorflow
from tensorflow.keras import utils
from tensorflow.keras.models import load_model, Model
import numpy as np
import cv2
from sklearn.metrics import multilabel_confusion_matrix, classification_report\
  ,accuracy_score, f1_score, precision_score, confusion_matrix
from IPython.display import Image, display
import matplotlib.pyplot as plt
import matplotlib.cm as cm

## Functions

In [60]:
def subset_accuracy(y_true, y_pred):
    return tensorflow.py_function(accuracy_score, (y_true, y_pred), tensorflow.double)
def load_model_data(model_config):
    """
    model_config(str): model configuration - either TL, LLR, or PL
    returns:
    tensorflow model (model) - Model for AU detection
    """
    if model_config == 'TL':
        model_path  = 'C:\\Users\\user\\Documents\\MSc Computer\\Cluster\\model_logs\\AU_Model_cluster_training\\au_00015_model_24102022_028\\model-86-0.00.hdf5'
    elif model_config == 'LLR':
        model_path = 'C:\\Users\\user\\Documents\\MSc Computer\\Cluster\\missinglabels\\model_logs\\00061-test-train-30122022\\model-376-0.67_LL-R.hdf5'
    elif model_config == 'PL':
        model_path = 'C:\\Users\\user\\Documents\\MSc Computer\\Cluster\\pseudo\\model_logs\\au_models\\00009-test-train-20112022\\model-60-0.00.hdf5'
    else:
        raise ValueError (f'{model_config}: Model configuration specified is incorrect.')
    if model_config == 'PL':
        return load_model(model_path,custom_objects={"subset_accuracy": subset_accuracy})
    else:
        return load_model(model_path)
def create_grad_cam_model(model_c):
    
    return gradCaM
def get_image_paths(dataframe_path):
    df = pd.read_csv(dataframe_path)
    return df['Path'].tolist()
def preprocess_image(image_path):
    image = utils.load_img(image_path)
    image = utils.img_to_array(image)
    preprocessed_image = np.expand_dims(image, axis=0)
    return preprocessed_image
def get_img_array(img_path, size):
    # `img` is a PIL image of size 224*224
    img = tensorflow.keras.utils.load_img(img_path, target_size=size)
    # `array` is a float32 Numpy array of shape (224, 224, 3)
    array = tensorflow.keras.utils.img_to_array(img)
    # We add a dimension to transform our array into a "batch"
    # of size (1, 224, 224, 3)
    array = np.expand_dims(array, axis=0)
    return array
def save_and_display_gradcam(img_path, heatmap, cam_path="cam.jpg", alpha=0.4):
    # Load the original image
    img = tensorflow.keras.utils.load_img(img_path)
    img = tensorflow.keras.utils.img_to_array(img)

    # Rescale heatmap to a range 0-255
    heatmap = np.uint8(255 * heatmap)

    # Use jet colormap to colorize heatmap
    jet = cm.get_cmap("jet")

    # Use RGB values of the colormap
    jet_colors = jet(np.arange(256))[:, :3]
    jet_heatmap = jet_colors[heatmap]

    # Create an image with RGB colorized heatmap
    jet_heatmap = tensorflow.keras.utils.array_to_img(jet_heatmap)
    jet_heatmap = jet_heatmap.resize((img.shape[1], img.shape[0]))
    jet_heatmap = tensorflow.keras.utils.img_to_array(jet_heatmap)

    # Superimpose the heatmap on original image
    superimposed_img = jet_heatmap * alpha + img
    superimposed_img = tensorflow.keras.utils.array_to_img(superimposed_img)

    # Save the superimposed image
    superimposed_img.save(cam_path)

    # Display Grad CAM
    display(Image(cam_path))


def overlay_image(heatmap, image, alpha=0.5, colormap = cv2.COLORMAP_VIRIDIS):
    heatmap = cv2.applyColorMap(heatmap,colormap)
    output = cv2.addWeighted(image, alpha, heatmap, 1-alpha, 0)
    return (heatmap, output)
    
    
    

## Load Model

In [76]:
# load model
model = load_model_data('PL')

In [75]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 vgg16 (Functional)          (None, 7, 7, 512)         14714688  
                                                                 
 flatten (Flatten)           (None, 25088)             0         
                                                                 
 final_au (Dense)            (None, 12)                301068    
                                                                 
Total params: 15,015,756
Trainable params: 15,015,756
Non-trainable params: 0
_________________________________________________________________


In [79]:
model.get_layer('vgg16').summary()

Model: "vgg16"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 224, 224, 3)]     0         
                                                                 
 block1_conv1 (Conv2D)       (None, 224, 224, 64)      1792      
                                                                 
 block1_conv2 (Conv2D)       (None, 224, 224, 64)      36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, 112, 112, 64)      0         
                                                                 
 block2_conv1 (Conv2D)       (None, 112, 112, 128)     73856     
                                                                 
 block2_conv2 (Conv2D)       (None, 112, 112, 128)     147584    
                                                                 
 block2_pool (MaxPooling2D)  (None, 56, 56, 128)       0     

In [67]:
grad = Model(inputs = 
            model.inputs,
            outputs = [model.get_layer('vgg16').get_layer('block5_conv3').output]
            )

ValueError: Graph disconnected: cannot obtain value for tensor KerasTensor(type_spec=TensorSpec(shape=(None, 224, 224, 3), dtype=tf.float32, name='input_1'), name='input_1', description="created by layer 'input_1'") at layer "block1_conv1". The following previous layers were accessed without issue: []

In [19]:
model.inputs

[<KerasTensor: shape=(None, 224, 224, 3) dtype=float32 (created by layer 'vgg16_input')>]

In [6]:
model.output

<KerasTensor: shape=(None, 12) dtype=float32 (created by layer 'final_au')>

In [11]:
for layers in model.layers:
    print(layers)

In [18]:
model.get_layer('vgg16').get_layer('block5_conv3').output

<KerasTensor: shape=(None, 14, 14, 512) dtype=float32 (created by layer 'block5_conv3')>

In [7]:
vgg16 = tensorflow.keras.applications.vgg16.VGG16()

553467096/553467096 [==============================] - 230s 0us/step


In [8]:
vgg16.summary()

Model: "vgg16"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 224, 224, 3)]     0         
                                                                 
 block1_conv1 (Conv2D)       (None, 224, 224, 64)      1792      
                                                                 
 block1_conv2 (Conv2D)       (None, 224, 224, 64)      36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, 112, 112, 64)      0         
                                                                 
 block2_conv1 (Conv2D)       (None, 112, 112, 128)     73856     
                                                                 
 block2_conv2 (Conv2D)       (None, 112, 112, 128)     147584    
                                                                 
 block2_pool (MaxPooling2D)  (None, 56, 56, 128)       0     

In [9]:
base_model = tensorflow.keras.applications.VGG16(
    include_top=False,
    weights="imagenet",
    input_shape=(224, 224, 3))

In [10]:
base_model.summary()

Model: "vgg16"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_2 (InputLayer)        [(None, 224, 224, 3)]     0         
                                                                 
 block1_conv1 (Conv2D)       (None, 224, 224, 64)      1792      
                                                                 
 block1_conv2 (Conv2D)       (None, 224, 224, 64)      36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, 112, 112, 64)      0         
                                                                 
 block2_conv1 (Conv2D)       (None, 112, 112, 128)     73856     
                                                                 
 block2_conv2 (Conv2D)       (None, 112, 112, 128)     147584    
                                                                 
 block2_pool (MaxPooling2D)  (None, 56, 56, 128)       0     

In [24]:
 tensorflow.keras.utils.plot_model(model, show_shapes=True, show_dtype=True)

You must install pydot (`pip install pydot`) and install graphviz (see instructions at https://graphviz.gitlab.io/download/) for plot_model/model_to_dot to work.


In [28]:
Model(
inputs = vgg16.inputs,
outputs=[vgg16.get_layer('block5_conv3').output, vgg16.output])